# Preprocess Data
Installing library such as nltk, tflearn, tensorflow, numpy

In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

!pip install tflearn
!pip install tensorflow
import numpy
import tflearn
import tensorflow as tf
import random

     |████████████████████████████████| 462 kB 23.4 MB/s 


# Import dataset
Dataset create on json format that contain of generate the bot nlp process

In [ ]:
import json
with open('intents.json') as file:
  data = json.load(file)

# Seperate data

In [ ]:
words = []
labels = []
docs_x = []
docs_y = []

## Extract the data

In [ ]:
for intent in data['intents']:
  for pattern in intent['patterns']:
    wrds = nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent["tag"])

  if intent['tag'] not in labels:
    labels.append(intent['tag'])

# Stemming Data

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w !="?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [ ]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
  bag = []

  wrds = [stemmer.stem(w.lower()) for w in doc]

  for w in words:
    if w in wrds:
      bag.append(1)
      else:
        bag.append(0)

  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1

  training.append(bag)
  output.append(output_row)

In [ ]:
training = numpy.array(training)
output = numpy.array(output)

# Model

In [ ]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Train & Save The Model

In [ ]:
model.fit(training, 
          output, 
          n_epoch=1000, 
          batch_size=8, 
          show_metric=True)
model.save("model.tflearn")